# Projeto 16: Geração de textos

Codificação adaptada da documentação do TensorFlow: https://www.tensorflow.org/beta/tutorials/text/text_generation




- Usaremos uma base de dados extraída de textos escritos por Shakespeare's
- Link: http://karpathy.github.io/2015/05/21/rnn-effectiveness/
- O objetivo é treinar a LSTM para prever o próximo caractere em uma sequência de texto


# Etapa 1: Importação das bibliotecas

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import os
import time
tf.__version__

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


'2.15.0'

# Etapa 2: Carregamento e exploração da base de dados

In [2]:
data_url = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
#Download the dataset

1115394/1115394 [==============================] - 0s 0us/step


In [3]:
dataset_text = open(data_url, 'rb').read().decode(encoding = 'utf-8') #Read the dataset

In [4]:
print(dataset_text) #Show the dataset content

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
len(dataset_text)

1115394

In [7]:
vocab = sorted(set(dataset_text)) #Take out repeatetives words. Keeping just one word for each repeated word

In [9]:
print('{} unique characters'.format(len(vocab)))#Number of characteres

65 unique characters


In [10]:
vocab

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

# Etapa 3: Mapeamento de texto para números

In [11]:
char2idx = {char: index for index, char in enumerate(vocab)} #Create a dictionary with index and characters

In [12]:
char2idx

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36,
 'Y': 37,
 'Z': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

In [13]:
idx2char = np.array(vocab)#Turn it to numpy array

In [14]:
idx2char

array(['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
       'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'],
      dtype='<U1')

In [15]:
idx2char[10]

':'

In [16]:
char2idx[':']

10

In [17]:
text_as_int = np.array([char2idx[char] for char in dataset_text]) #Convert each character in a index and put it inside a numpy array

In [19]:
text_as_int

array([18, 47, 56, ..., 45,  8,  0])

In [20]:
text_as_int.shape

(1115394,)

In [21]:
print('{} characters mapped to int ---> {}'.format(repr(dataset_text[:13]), text_as_int[:13])) #Map the character and index variables

'First Citizen' characters mapped to int ---> [18 47 56 57 58  1 15 47 58 47 64 43 52]


# Etapa 4: Criação dos exemplos de treinamento e batches


- Dividiremos a base de dados em uma sequência de caracteres com "seq_length"
- A saída (dados reais) será o mesmo que a entrada, porém, com um caractere deslocado
- Exemplo com o texto "Hello" e seq_len = 4
    - Entrada: "Hell"
    - Saída: "ello"


In [22]:
len(dataset_text) #Register amount

1115394

In [23]:
seq_length = 100 #sequency will have 100 characters
examples_per_epoch = len(dataset_text) // seq_length
examples_per_epoch #Amount of register we will work in each epoch

11153

In [24]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int) #Convert it to a Tensorflow variable

In [25]:
char_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

In [28]:
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)#Divide the dataset in batches

In [29]:
sequences

<_BatchDataset element_spec=TensorSpec(shape=(101,), dtype=tf.int64, name=None)>

In [27]:
for item in sequences.take(50):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'
'zens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but th'
'e superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we a'
're too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particula'
'rise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we bec'
'ome rakes: for the gods kn

In [30]:
def split_input_target(chunk): #Divide input and output
  input_text = chunk[:-1]#Get all the registers less the last
  target_text = chunk[1:]#Get all the registers less the first one
  return input_text, target_text #Return it

In [31]:
dataset = sequences.map(split_input_target) #Apply the function split_input_target in each element inside the sequences array

In [32]:
#Show the result after the process
for input_example, target_example in dataset.take(10):#input_example=> Neural Network input | target_example=> The goal(output)
  print('Input data:', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data: 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
Input data: 'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you '
Target data: 're all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
Input data: "now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us k"
Target data: "ow Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
Input data: "ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be "
Target data: "l him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
Input data: 'one: away, away!\

In [34]:
batch_size = 64
buffer_size = 10000 #Defines the Neural network will be trained just with 10000 registers

In [35]:
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder = True)

In [36]:
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

# Etapa 5: Construção do modelo

In [37]:
len(vocab)

65

In [38]:
vocab_size = len(vocab)

In [39]:
embedding_dim = 256 #Each word will be maped in 256 columns values

In [40]:
rnn_units = 1024

In [42]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size): #Function to build the neural network
  model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]), #Apply the words mapping to the embeddig matrix
                               tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'), # Create a LSTM layer
                               tf.keras.layers.Dense(vocab_size)]) #Create a conventional dense layer
  return model #Return the neural network

In [43]:
model = build_model(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=batch_size) #Build

In [44]:
for input_example_batch, target_example_batch in dataset.take(10): #input_example_batch=> Neural Network input | target_example_batch=> The goal(output)
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape) #(batch_size, sequence_length, vocabsize)

(64, 100, 65)
(64, 100, 65)
(64, 100, 65)
(64, 100, 65)
(64, 100, 65)
(64, 100, 65)
(64, 100, 65)
(64, 100, 65)
(64, 100, 65)
(64, 100, 65)


In [49]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1) #get the caracters that was predicted for the first text

In [50]:
sampled_indices

<tf.Tensor: shape=(100, 1), dtype=int64, numpy=
array([[59],
       [36],
       [38],
       [54],
       [ 4],
       [28],
       [15],
       [ 9],
       [31],
       [ 2],
       [52],
       [10],
       [ 5],
       [56],
       [ 8],
       [46],
       [42],
       [29],
       [15],
       [50],
       [56],
       [57],
       [29],
       [32],
       [45],
       [15],
       [59],
       [51],
       [10],
       [25],
       [48],
       [61],
       [45],
       [58],
       [60],
       [29],
       [15],
       [50],
       [24],
       [ 7],
       [38],
       [17],
       [ 9],
       [57],
       [34],
       [63],
       [50],
       [55],
       [14],
       [50],
       [23],
       [11],
       [14],
       [23],
       [55],
       [57],
       [33],
       [62],
       [18],
       [ 6],
       [34],
       [60],
       [36],
       [44],
       [52],
       [19],
       [57],
       [55],
       [49],
       [43],
       [10],
       [ 1],
       [36],
   

In [51]:
sampled_indices = tf.squeeze(sampled_indices, axis = -1).numpy() #Transforma this matrix to a numpy array

In [52]:
sampled_indices

array([59, 36, 38, 54,  4, 28, 15,  9, 31,  2, 52, 10,  5, 56,  8, 46, 42,
       29, 15, 50, 56, 57, 29, 32, 45, 15, 59, 51, 10, 25, 48, 61, 45, 58,
       60, 29, 15, 50, 24,  7, 38, 17,  9, 57, 34, 63, 50, 55, 14, 50, 23,
       11, 14, 23, 55, 57, 33, 62, 18,  6, 34, 60, 36, 44, 52, 19, 57, 55,
       49, 43, 10,  1, 36, 48, 59, 11, 43,  6, 33,  7, 63, 33, 18, 20, 43,
       38, 43,  6,  4, 61, 26,  2, 11, 11, 11,  4, 57, 24, 53, 12])

In [53]:
print('Input: \n', repr(''.join(idx2char[input_example_batch[0]])))
print()
print('Next char predictions: \n', repr(''.join(idx2char[sampled_indices])))

Input: 
 "k. His pupil age\nMan-enter'd thus, he waxed like a sea,\nAnd in the brunt of seventeen battles since\n"

Next char predictions: 
 "uXZp&PC3S!n:'r.hdQClrsQTgCum:MjwgtvQClL-ZE3sVylqBlK;BKqsUxF,VvXfnGsqke: Xju;e,U-yUFHeZe,&wN!;;;&sLo?"


# Etapa 6: Treinamento do modelo

### Otimizador e loss function

In [54]:
def loss(labels, logits): #loss function
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [55]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)

In [56]:
example_batch_loss.numpy().mean()

4.1746006

In [57]:
model.compile(optimizer='Adam', loss=loss)

### Checkpoints

In [58]:
#Create training_checkpoints folder
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

### Execução do treinamento

In [59]:
epochs = 10
history = model.fit(dataset, epochs = epochs, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 17s 66ms/step - loss: 2.5885
Epoch 2/10
172/172 [==============================] - 13s 67ms/step - loss: 1.9014
Epoch 3/10
172/172 [==============================] - 13s 66ms/step - loss: 1.6449
Epoch 4/10
172/172 [==============================] - 13s 68ms/step - loss: 1.5069
Epoch 5/10
172/172 [==============================] - 13s 69ms/step - loss: 1.4259
Epoch 6/10
172/172 [==============================] - 13s 69ms/step - loss: 1.3692
Epoch 7/10
172/172 [==============================] - 14s 70ms/step - loss: 1.3242
Epoch 8/10
172/172 [==============================] - 15s 70ms/step - loss: 1.2865
Epoch 9/10
172/172 [==============================] - 15s 70ms/step - loss: 1.2510
Epoch 10/10
172/172 [==============================] - 14s 70ms/step - loss: 1.2157


# Etapa 7: Geração de textos

### Restauração do último checkpoint

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size = 1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

### Loop de previsão

In [ ]:
def generate_text(model, start_string):
  # Número de caracteres a serem gerados
  num_generate = 1000

  # Conversão dos caracteres iniciais de string para números
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Lista para armazenar os textos gerados pela rede neural
  text_generated = []

  # Parâmetro temperatura
  # Valores baixos resultam em melhores textos (deve ser testado)
  temperature = 1.0

  # Loop para gerar os textos
  for i in range(num_generate):
    # Previsões
    predictions = model(input_eval)

    # Tratamento das previsões
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # Passamos a previsão como próxima entrada da rede
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string='ROMEO: '))